# ПостНаука

In [740]:
import artm
import hierarchy_utils
import os
import glob
import sys
import numpy as np
import sklearn

In [741]:
data_path_UCI = 'E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/'
data_path_batches = 'E:/lib/ARTM/htest/UCI_collections/batches/'
if len(glob.glob(data_path_batches + "/*.batch")) < 1:
    batch_vectorizer = artm.BatchVectorizer(collection_name='pn', data_format='bow_uci', \
                data_path = data_path_UCI, batch_size = 225, target_folder=data_path_batches)
else:
    batch_vectorizer = artm.BatchVectorizer(data_path=data_path_batches, data_format='batches')

In [742]:
class_ids={'text':1.0, 'flat_tag':10.0, 'author': 5.0}


In [743]:
# quality measures of models
scores_list = []
"""
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreText',
                                         class_id='ngramm',
                                         #topic_names=topics_subj
                                        ))
scores_list.append(artm.SparsityThetaScore(name='SparsityScoreThetaSubj',
                                           #topic_names=topics_subj
                                          ))

scores_list.append(artm.SparsityPhiScore(name='SparsityScoreAuthor', class_id='author'))
scores_list.append(artm.SparsityPhiScore(name='SparsityScoreTag', class_id='flat_tag'))
scores_list.append(artm.TopicKernelScore(name='TopicKernelText', class_id='text', probability_mass_threshold=0.25))
"""

scores_list.append(artm.TopTokensScore(name='TopTokensScoreText', class_id='text', num_tokens=15))
scores_list.append(artm.TopTokensScore(name='TopTokensTag', class_id='flat_tag', num_tokens=10))
scores_list.append(artm.PerplexityScore(name='PerplexityScore', class_ids=['text']))
scores_list.append(artm.PerplexityScore(name='PerplexityScore_Tag', class_ids=['flat_tag']))

In [744]:
dict_path = 'E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/dictionary.txt'
if not os.path.isfile(dict_path):
    dictionary = artm.Dictionary('dictionary')
    dictionary.gather(data_path=batch_vectorizer.data_path, 
                      vocab_file_path='E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/vocab.pn.txt' )
    dictionary.save_text(dict_path)
else:
    dictionary = artm.Dictionary('dictionary')
    dictionary.load_text(dict_path)

### Тут строится 0 уровень

In [745]:
topic_names0=['topic_'+str(i) for i in range(0,20)]

In [746]:
regularizers_list = []


#сглаживание фоновой темы по словам
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagRegBackground',class_ids=['text'],
                                                         topic_names=topic_names0[-1:],tau=10000)) 
#декоррелирование остальных тем по словам
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],
                                                         topic_names=topic_names0[:-1],
                                                         tau=5000000))


"""regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],
                                                         topic_names=topic_names0[:-1],tau=-15))
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         #topic_names=topics_subj,
                                                         tau=0.01))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj,
                                                           tau=0.01))"""

"regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiTagReg',class_ids=['flat_tag'],tau=500000))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],\n                                                         topic_names=topic_names0[:-1],tau=-15))\nregularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',\n                                                         class_ids=['text'],\n                                                         #topic_names=topics_subj,\n                                                         tau=0.01))\n\n#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',\n                                                           #topic_names=topics_subj,\n                                                           tau=0.01))"

In [747]:
%%time

topicNum0 = 10

hier = hierarchy_utils.hARTM(class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

Wall time: 189 ms


In [748]:
level0 = artm.ARTM(topic_names=topic_names0, class_ids=class_ids,
             regularizers=regularizers_list,
             scores=scores_list, cache_theta=True,
             seed=37)

In [749]:
%%time

level0.initialize(dictionary=dictionary)
level0.fit_offline(batch_vectorizer, num_collection_passes=10)

Wall time: 24.1 s


In [750]:
output = ""
tokens = level0.score_tracker["TopTokensTag"].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: история история_россии христианство россия русь средневековье филология религия кавказ рим 
topic_1: медицина биология генетика геном ген днк онкология клетка биомедицина биоинформатика 
topic_2: физика физика_элементарных_частиц квантовая_физика космология вселенная элементарная_частица бозон_хиггса большой_адронный_коллайдер темная_материя кварк 
topic_3: искусство культурология театроведение театр платон ислам религия искусствоведение политическая_философия культура 
topic_4: астрономия астрофизика звезды космос галактика вселенная солнце солнечная_система планета нейтронные_звезды 
topic_5: кино кинематограф история_россии сталин_иосиф ссср история массовая_культура археология культурология великая_отечественная_война 
topic_6: политика социология право государство город общество россия политология экономика юриспруденция 
topic_7: биология микробиология математика белки клетка бактерии эволюция рнк генетика молекула 
topic_8: культура философия история литература средневе

In [751]:
output = ""
tokens = level0.score_tracker['TopTokensScoreText'].last_tokens
for topic_name in level0.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

topic_0: русский земля церковь имя христианский народ римский святой царь король русь бог император империя великий 
topic_1: клетка ген организм белок днк заболевание болезнь генетический пациент мутация препарат лечение нервный ткань геном 
topic_2: частица энергия поле физика магнитный волна электрон кварк излучение материя атом свет взаимодействие фотон эксперимент 
topic_3: китайский театр тирания платон бильгамес тиран пьеса артур ислам египет тональный письменность египетский джихад толкин 
topic_4: объект находиться звезда галактика солнце космический солнечный телескоп диск нейтронный звездный газ орбита астрономия спутник 
topic_5: фильм кино сталин pcourse num кинематограф святилище культовый зритель фигурка декабрист режиссер бронзовый перлюстрация ведьма 
topic_6: политический общество город право государство власть политика закон свобода общественный партия городской понятие государственный правовой 
topic_7: работать многий количество использовать основной достаточно дан

### Перплексия 0

In [752]:
print(level0.score_tracker['PerplexityScore'].value[-1])
print(level0.score_tracker['PerplexityScore_Tag'].value[-1])

7575.11196155576
111.9541470700059


In [753]:
level0.get_phi(class_ids=['text', 'flat_tag'])

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
признаваться,1.413294e-04,9.663365e-11,8.216170e-08,1.617626e-05,3.103081e-06,7.052704e-07,3.089687e-04,1.731463e-07,1.804565e-04,6.496241e-05,3.391317e-05,9.097809e-08,1.667806e-05,0.000000e+00,1.097560e-04,8.441778e-05,5.974006e-05,1.448105e-07,4.128140e-05,1.289605e-05
734,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.086517e-16,0.000000e+00,4.101181e-06,7.061508e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.669669e-14,0.000000e+00,1.974618e-12,2.057319e-16,9.519392e-06,0.000000e+00,0.000000e+00,1.289318e-05
протекать,8.042396e-06,2.592441e-04,7.575507e-05,2.251052e-05,3.930211e-05,0.000000e+00,6.446349e-05,2.506969e-05,5.633919e-06,7.537249e-05,9.505470e-07,7.021845e-05,0.000000e+00,2.772115e-04,4.986586e-05,1.331610e-06,3.620377e-05,3.462243e-06,3.150120e-05,1.289659e-05
люцерна,8.978740e-14,4.998699e-06,0.000000e+00,0.000000e+00,2.312807e-06,0.000000e+00,2.736519e-13,1.226904e-10,1.590152e-11,8.401204e-12,3.778369e-13,0.000000e+00,0.000000e+00,6.614572e-07,6.835898e-07,0.000000e+00,4.353089e-09,3.532997e-11,3.308968e-07,1.289527e-05
слалом,4.292128e-16,0.000000e+00,2.526564e-11,0.000000e+00,1.227026e-05,0.000000e+00,0.000000e+00,2.276963e-15,0.000000e+00,4.231652e-16,8.678598e-15,5.756335e-07,0.000000e+00,1.725171e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.289281e-05
дрессура,3.986164e-15,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.262019e-09,0.000000e+00,6.762039e-07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.917162e-09,7.685178e-16,1.601589e-14,8.087025e-05,1.776639e-08,1.289371e-05
logica,0.000000e+00,0.000000e+00,0.000000e+00,4.026599e-11,0.000000e+00,0.000000e+00,0.000000e+00,2.790901e-16,5.850395e-06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.289279e-05
анализировать,2.707020e-11,2.098057e-04,4.177224e-05,6.337878e-05,3.409155e-05,7.042935e-05,6.263110e-05,2.425215e-04,4.121169e-04,3.804425e-04,9.006775e-06,1.494845e-05,4.329264e-06,9.488508e-05,1.716564e-04,6.356605e-04,4.755143e-07,7.500147e-05,3.127985e-04,1.289704e-05
matthias,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.781385e-07,0.000000e+00,2.766708e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.887134e-13,0.000000e+00,1.684012e-10,0.000000e+00,2.117693e-16,1.289358e-05
аналитический,1.442429e-13,1.492121e-06,7.741464e-05,0.000000e+00,9.011520e-06,0.000000e+00,5.876542e-06,6.448050e-05,8.157463e-04,1.458308e-04,3.306142e-04,5.508320e-08,3.267034e-04,5.930525e-06,8.818661e-05,6.773213e-04,3.999195e-07,0.000000e+00,2.832238e-04,1.289639e-05


### Стоп-слова 0

In [754]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level0.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level0.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 3):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_0.txt', 'w')
f.write(stopwords)
f.close()

In [755]:
hier._levels.append(level0)

### Модификация value для словаря

In [756]:
#создает словарь, в котором для слов из tokens[topic_name] value=1, а для остальных слов value=0
def create_my_dict(tokens, topic_name):
    line='newline'
    with open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/dictionary.txt', 'r', encoding='utf-8', errors='replace') as dict0, open('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', 'w+') as my_dict:
        while (line!=''):
            try:
                line=dict0.readline()
                line=line.split(', ')
                if(len(line)==5):
                    if (line[0] in tokens[topic_name]):
                        line[1] = '1.0'
                    else:
                        line[1] = '0.0'
                line = ', '.join(line)
                my_dict.write(line)
            except UnicodeEncodeError:
                pass
            except UnicodeDecodeError:
                pass
    my_dict = artm.Dictionary('mydict'+topic_name[-2:])
    my_dict.load_text('E:/lib/ARTM/htest/UCI_collections/UCI_trimmed/my_dict'+topic_name[-2:]+'.txt', encoding = 'cp1251')
    print(my_dict)
    return my_dict
    

### Тут строится 1 уровень

In [757]:
hier.del_level(1)

In [758]:
hier._levels

[artm.ARTM(num_topics=20, num_tokens=79402, class_ids=['text', 'flat_tag', 'author'])]

In [759]:
topic_names1=['topic_'+str(i) for i in range(0,80)]

In [ ]:
regularizers_list = []

#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1', class_ids=['text'],
#                                                topic_names=['topic_'+str(i) for i in range(0,60)],
#                                                         tau=5000000))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
#                                                topic_names=['topic_'+str(i) for i in range(0,60)],
#                                                        tau=-100))

#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiRegB', class_ids=['text'],
#                                                topic_names=['topic_'+str(i) for i in range(60,80)],
#                                                         tau=50000))

#for i in range(60,79):
#    regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextRegB'+str(i),
#                           class_ids=['text', 'flat_tag'],
#                           topic_names=['topic_'+str(i)], 
#                           dictionary = create_my_dict(tokens=level0.score_tracker['TopTokensTag'].last_tokens,
#                                                       topic_name='topic_'+str(i-60)),
#                           tau=15000))
    
regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextRegBackground',
                            class_ids=['text', 'flat_tag'],
                            topic_names=topic_names1[-1:], 
                            tau=5000))

In [ ]:
%%time

topicNum1 = 80

level1 = hierarchy_utils.ARTM_Level(level0, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum1,
                                        topic_names=topic_names1,
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [ ]:
level1.regularizers

In [ ]:
%%time

level1.initialize(dictionary = dictionary)
level1.fit_offline(batch_vectorizer, num_collection_passes = 5)

In [ ]:
tokens = level1.score_tracker["TopTokensTag"].last_tokens
output = ''
for topic_name in level1.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

In [ ]:
tokens = level1.score_tracker["TopTokensScoreText"].last_tokens
output = ''
for topic_name in level1.topic_names:
    output += topic_name + ': '
    for word in tokens[topic_name]:    
        output += word + ' '
    output += '\n'
print(output)

### Перплексия 1

In [ ]:
print(level1.score_tracker['PerplexityScore'].value[-1])
print(level1.score_tracker['PerplexityScore_Tag'].value[-1])

### Стоп-слова 1

In [ ]:
tokens_list = []
unique_tokens = []
not_unique = []
stopwords = ''
for topic_name in level1.topic_names:
    unique_tokens = list(set(unique_tokens+tokens[topic_name]))
    tokens_list += tokens[topic_name]
    
counter = {word: 0 for word in unique_tokens}
for word in unique_tokens:
    for topic_name in level1.topic_names:
        if word in tokens[topic_name]:
            counter[word] +=1

for (word, n) in counter.items():
    if (n >= 4):
        stopwords += word + ' '
print(stopwords)

f = open('stopwords_1.txt', 'w')
f.write(stopwords)
f.close()

### Подтемы

In [ ]:
psi1 = level1.get_psi()

In [ ]:
threshold = 0.05
psi1  # p(subtopic|topic)

In [ ]:
tokens0 = level0.score_tracker["TopTokensTag"].last_tokens
tokens1 = level1.score_tracker["TopTokensTag"].last_tokens

In [ ]:
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if (psi1.values[s, t] > threshold):
        #&(s<60):
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

related_topics = 0
for row in (psi1.values>threshold):
    if (sum(row) != 0):
        related_topics += 1

print("\n child topics: ", related_topics)    

In [737]:
tokens0 = level0.score_tracker["TopTokensScoreText"].last_tokens
tokens1 = level1.score_tracker["TopTokensScoreText"].last_tokens

In [738]:
output = ''
related = 0
for t, topic_name in enumerate(level0.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level1.topic_names):
        if psi1.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edge: ", related)

 topic_0: русский земля церковь имя христианский народ римский святой царь король русь бог император империя великий 
 topic_1: клетка ген организм белок днк заболевание болезнь генетический пациент мутация препарат лечение нервный ткань геном 
    topic_32: клетка медицина организм белок геном вирус структура число появляться работать достаточно использовать нервный генетика генетический 
    topic_56: клетка медицина организм генетика геном материал мозг генетический структура бактерия соединение технология молекулярный эволюция мутация 
 topic_2: частица энергия поле физика магнитный волна электрон кварк излучение материя атом свет взаимодействие фотон эксперимент 
    topic_11: частица энергия поле электрон волна кварк модель магнитный черный момент данный количество свет использовать структура 
    topic_46: частица электрон кварк энергия магнитный поле вещество квантовый материя свет волна сверхпроводимость космология модель атом 
    topic_74: энергия частица атом находиться 

In [739]:
hier._levels.append(level1)

### Построение третьего уровня

In [ ]:
hier.del_level(2)

In [ ]:
hier._levels

In [ ]:
regularizers_list = []

#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiAuthorReg',class_ids=['author'],tau=0.05))
#regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTagReg',class_ids=['flat_tag'],tau=-1))
#regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg',class_ids=['text'],tau=500000000))
regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg1',
                                                         tau=50000000))

regularizers_list.append(artm.DecorrelatorPhiRegularizer(name='DecorrPhiReg2',
                                                         tau=50000))

regularizers_list.append(artm.SmoothSparsePhiRegularizer(name='SPPhiTextReg',
                                                         class_ids=['text'],
                                                         tau=5000))

#regularizers_list.append(artm.SmoothSparseThetaRegularizer(name='SmoothThetaRegSubj',
                                                           #topic_names=topics_subj, tau=100))

In [ ]:
%%time

topicNum2 = 50

level2 = hierarchy_utils.ARTM_Level(level1, phi_batch_weight=10.0**3, 
                                        num_topics=topicNum2,  
                                         class_ids=class_ids,
                                          regularizers=regularizers_list,
                                           scores=scores_list,
                                            num_document_passes=1, 
                                             cache_theta=True, seed=124)

In [ ]:
%%time

level2.initialize(dictionary=dictionary)
level2.fit_offline(batch_vectorizer, num_collection_passes=10)

In [ ]:
psi2 = level2.get_psi()
psi2>0.05

In [ ]:
output = ""
tokens2 = level2.score_tracker["TopTokensTag"].last_tokens
for topic_name in level2.topic_names:
    output += topic_name + ': '
    for word in tokens2[topic_name]:    
        output += word + " "
    output += "\n"
print(output)

In [ ]:
tokens0 = level1.score_tracker["TopTokensTag"].last_tokens
tokens1 = level2.score_tracker["TopTokensTag"].last_tokens

In [ ]:
threshold = 0.05

In [ ]:
output = ''
related = 0
for t, topic_name in enumerate(level1.topic_names):
    output += topic_name + ': '
    for word in tokens0[topic_name]:    
        output += word + ' '
    print("\b", output)
    output=''
    for s, topic_name1 in enumerate(level2.topic_names):
        if psi2.values[s, t] > threshold:
            related += 1
            output += "    "+ topic_name1 + ': '
            for word in tokens1[topic_name1]:    
                output += word + ' '
            print (output)
            output =''
print("\n edges: ", related)

### Сохронение

In [ ]:
hier._levels

In [ ]:
import pickle

In [ ]:
%%time

# OUR OWN VERY COOL SAVE

dump = {}

for level_idx, level in enumerate(hier._levels):
    if level_idx > 0:
        # Zeroth level hierarchy doesn't have get_psi()
        dump["psi_%d" % level_idx] = level.get_psi()
    dump["phi_%d" % level_idx] = level.get_phi()
dump["theta"] = hier.get_theta()

In [ ]:
pickle.dump(dump, open("hartm.mdl", "wb"))